<a href="https://colab.research.google.com/github/younes-sadi/semantic-search-task2-LSP2431549/blob/main/Belkacem_Sadi_Task2_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Belkacem Sadi LSP_Id: 2431549 Task_2

This project uses Retrieval-Augmented Generation (RAG) by embedding 5,000 real news articles using a transformer (MiniLM), storing the vectors in FAISS, retrieving top-k relevant entries for a user query, and generating a final answer using T5. The pipeline mimics the internal architecture of models like facebook/rag-sequence-nq while remaining lightweight and Colab-friendly

In [ ]:
!pip install -q datasets==2.16.0
!pip install -q transformers sentence-transformers faiss-cpu


In [ ]:
!pip install -q datasets==2.16.0
!pip install -q transformers sentence-transformers faiss-cpu

from datasets import load_dataset

dataset = load_dataset("ag_news", split="train[:5000]")
texts = [x["text"] for x in dataset]


In [ ]:

import pandas as pd

df = pd.DataFrame({"text": texts})
df.to_csv("ag_news_5000.csv", index=False)


In [ ]:

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

encoder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = encoder.encode(texts, convert_to_numpy=True, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [ ]:

from transformers import T5ForConditionalGeneration, T5Tokenizer

#  small generator model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
generator = T5ForConditionalGeneration.from_pretrained("t5-small")

query = "What are the latest innovations in AI?"
query_embedding = encoder.encode([query], convert_to_numpy=True)
D, I = index.search(query_embedding, k=3)

# Combine top 3 results into a context
context = " ".join(texts[i] for i in I[0])
prompt = f"question: {query} context: {context}"

inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
outputs = generator.generate(**inputs)
print("=== Answer ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

#  t5-large
tokenizer = T5Tokenizer.from_pretrained("t5-large")
generator = T5ForConditionalGeneration.from_pretrained("t5-large")

query = "What are the latest news of sport "

query_embedding = encoder.encode([query], convert_to_numpy=True)

D, I = index.search(query_embedding, k=5)

# top 5
context = " ".join(texts[i] for i in I[0])

prompt = f"question: {query} context: {context}"

# 7. Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

outputs = generator.generate(
    input_ids=inputs["input_ids"],
    max_length=300,
    min_length=50,
    length_penalty=1.2,
    num_beams=4,
    early_stopping=True
)

# 9. Decode
print("=== Answer ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
!git clone https://github.com/younes-sadi/semantic-search-task2-LSP2431549.git
!cd semantic-search-task2-LSP2431549


In [ ]:
!pip install --upgrade ipywidgets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd /content/semantic-search-task2-LSP2431549
!git add Belkacem_Sadi_Task2_DL.ipynb
!git commit -m "Upload task2"
!git push
